In [1]:
# Build Baseline dataset from all valid trials from Baseline sessions in the AFIRM and BAA classifications
# File structure from root_dir:
# {root_dir}/
# ├── {classification}/
# │   ├── {subject}/
# │   │   ├── {session}/
# │   │   │   ├── {trial}.c3d
# │   │   │   └── ...
# │   │   └── ...
# │   └── ...
# └── ...

root_dir = 'data'
classifications = ['AFIRM', 'BAA']
session = 'Baseline'

from utils import scandir_regex
from rat_trial import RatTrial
import os
import numpy as np

file_pattern = f"{root_dir}/({'|'.join(classifications)})/.*/{session}/Static.*\\.c3d"
print(f"Searching for files with pattern: {file_pattern}")

baseline_files = scandir_regex(root_dir, file_pattern)
for file in baseline_files:
    print(f"Processing file: {file}")
    trial = RatTrial.from_c3d_file(file)
    # Validate
    if not trial.validate_trial():
        print(f"Invalid trial: {file}")
        continue
    trial.to_trc(file.replace('.c3d', '.trc'), 
             rotation = np.array([[1, 0, 0],
                                  [0, 0, 1],
                                  [0, -1, 0]])
             )
    trial.scale_opensim_model(
        unscaled_model_path='opensim/Bilateral/FinalBilateral.osim',
        marker_set_path='opensim/Bilateral/bilateral_rodent_hindlimb_markerSet.xml',
        marker_file_name= os.path.basename(file).replace('.c3d', '.trc'),
        output_dir=os.path.dirname(file),
        scale_setup_path='opensim/Bilateral/generic_scale_setup_bilateral.xml'
    )


Searching for files with pattern: data/(AFIRM|BAA)/.*/Baseline/Static.*\.c3d
Processing file: data/BAA/BAA01/Baseline/Static01.c3d
[info] Processing subject Static01...
[info] Step 1: Loading generic model
[info] Updating Model file from 40000 to latest format...
[info] Loaded model FinalBilateral from file /home/hudson/Rats_Python/opensim/Bilateral/FinalBilateral.osim
[info] Loading marker set from '/home/hudson/Rats_Python/data/BAA/BAA01/Baseline//home/hudson/Rats_Python/opensim/Bilateral/bilateral_rodent_hindlimb_markerSet.xml'.
[info] Updated markers in model FinalBilateral
[info] Step 2: Scaling generic model
[info] Loaded marker file /home/hudson/Rats_Python/data/BAA/BAA01/Baseline/Static01.trc (12 markers, 676 frames)
[info] Measurement 'spine'
[info] 	pair 0 (TAIL, SPL6): model = 0.0393357, experimental = 0.0352886
[info] 	overall scale factor = 0.897113
[info] Measurement 'pelvis_length'
[info] 	pair 0 (RASI, RHIP): model = 0.0299833, experimental = 0.0221034
[info] 	pair 1 (L

In [ ]:
from rat_trial import RatTrial
import numpy as np
from utils import parse_enf_file
import polars as pl
from scipy.signal import butter, filtfilt

trial = RatTrial.from_c3d_file('data/BAA/BAA01/Baseline/Walk05.c3d')
trial.link_file('marker_model', 'data/BAA/BAA01/Baseline/Static01_marker.osim')
# Filter markers with 15 Hz low-pass filter


trial.to_trc('data/BAA/BAA01/Baseline/Walk05.trc', 
             rotation = np.array([[1, 0, 0],
                                  [0, 0, 1],
                                  [0, -1, 0]])
             )
trial.run_opensim_ik(model_path='data/BAA/BAA01/Baseline/Static01_marker.osim', 
                     trc_path=trial.get_linked_file("trc"), 
                     ik_setup_path='opensim/Bilateral/generic_Rat_Setup_IK.xml', 
                     output_dir='data/BAA/BAA01/Baseline',
                     start_time=trial.events[0].get_time(trial.points.rate),
                     end_time=trial.events[-1].get_time(trial.points.rate)
                     )

# Read .enf file to determine applied bodies
enf_data = parse_enf_file('data/BAA/BAA01/Baseline/Walk05.Trial.enf')
# Find keys that match the pattern 'fp%d' where %d is a digit and extract the digit and value
contacted_fp = {}
for key, value in enf_data.items():
    if key.lower().startswith('fp') and key[2:].isdigit():
        contacted_fp[int(key[2:])] = 'foot_l' if str(value).lower() == 'left' else 'foot_r' if str(value).lower() == 'right' else value
        
# Filter the force plate data with a 50 Hz low-pass filter and a notch filter from 58 to 62 Hz
# Then zero data outside of the events 
nyquist_freq = 0.5 * trial.analogs.rate
lowpass_cutoff = 50.0 / nyquist_freq 
low = butter(2, lowpass_cutoff, btype='low')
if low is None:
    raise ValueError("Low-pass filter coefficients could not be computed.")
notch_freq = [58.0 / nyquist_freq, 62.0 / nyquist_freq] 
notch = butter(2, notch_freq, btype='bandstop')
if notch is None:
    raise ValueError("Notch filter coefficients could not be computed.")
all_events = trial.get_events()
fp_rate = trial.analogs.rate
event_start_frame = all_events[0].get_frame(fp_rate)
event_end_frame = all_events[-1].get_frame(fp_rate)
for fp in trial.force_platforms:
    raw_data = fp.data.to_numpy()  # Convert to numpy for filtering
    filtered_data = filtfilt(low[0], low[1], raw_data, axis=0)
    filtered_data = filtfilt(notch[0], notch[1], filtered_data, axis=0)
    # Zero data outside of the events
    # Assign filtered data only within the event window, otherwise zero
    filtered_data[:event_start_frame, :] = 0
    filtered_data[event_end_frame:, :] = 0
    fp.data = pl.DataFrame(filtered_data, schema=fp.data.schema)
    
trial.export_force_platforms(output_dir='data/BAA/BAA01/Baseline',
                             applied_bodies=contacted_fp,
                             rotation=np.array([[1, 0, 0],
                                                [0, 0, 1],
                                                [0, -1, 0]])
                            )

trial.run_opensim_id(model_path=trial.get_linked_file('marker_model'), 
                     ik_results_path=trial.get_linked_file('ik_results'),
                     filter_cutoff=15.0,
                     external_loads_file=trial.get_linked_file('fp_setup'),
                     output_dir='data/BAA/BAA01/Baseline',
                     excluded_forces=['Muscles']
                    )
                     
# Pickle the trial object for later use
import pickle
with open('data/BAA/BAA01/Baseline/Walk05.trial.pkl', 'wb') as f:
    pickle.dump(trial, f)


2025-06-24 11:44:05.588 | WARNING  | trial:export_force_platforms:870 - Force platform 1 does not have an applied body defined. Skipping.
2025-06-24 11:44:05.588 | WARNING  | trial:export_force_platforms:870 - Force platform 2 does not have an applied body defined. Skipping.
2025-06-24 11:44:05.589 | WARNING  | trial:export_force_platforms:896 - Force platform 3 position unit mm does not match output unit m. Converting positions.
2025-06-24 11:44:05.589 | WARNING  | trial:export_force_platforms:900 - Force platform 3 torque unit Nmm does not match output unit Nm. Converting torques.
2025-06-24 11:44:05.591 | WARNING  | trial:export_force_platforms:896 - Force platform 4 position unit mm does not match output unit m. Converting positions.
2025-06-24 11:44:05.591 | WARNING  | trial:export_force_platforms:900 - Force platform 4 torque unit Nmm does not match output unit Nm. Converting torques.
2025-06-24 11:44:05.588 | WARNING  | trial:export_force_platforms:870 - Force platform 2 does no

[info] Loaded model Static01_marker from file /home/hudson/Rats_Python/data/BAA/BAA01/Baseline/Static01_marker.osim
[warning] Couldn't find file 'rat_spine.vtp'.
[warning] Couldn't find file 'rat_pelvis_r.vtp'.
[warning] Couldn't find file 'rat_femur_r.vtp'.
[warning] Couldn't find file 'rat_tibia_r.vtp'.
[warning] Couldn't find file 'rat_foot_r.vtp'.
[warning] Couldn't find file 'rat_pelvis_l_new_m.obj'.
[warning] Couldn't find file 'rat_femur_l.vtp'.
[warning] Couldn't find file 'rat_tibia_l.vtp'.
[warning] Couldn't find file 'rat_foot_l.vtp'.

               MODEL: Static01_marker
         coordinates: 22
              forces: 76
           actuators: 76
             muscles: 76
            analyses: 0
              probes: 0
              bodies: 9
              joints: 9
         constraints: 0
             markers: 12
         controllers: 0
  contact geometries: 0
misc modelcomponents: 0
[info] Running tool Walk05.
[info] Frame 574 (t = 2.87):	 total squared error = 0.000215102,

In [1]:
## Spline dataframe to stance and swing for one side
from rat_trial import RatTrial
from utils import sto_to_df
from scipy.interpolate import CubicSpline
import polars as pl
import numpy as np

trial = RatTrial.from_pkl('data/BAA/BAA01/Baseline/Walk05.trial.pkl')

df, _ = sto_to_df('data/BAA/BAA01/Baseline/Walk05_id.sto')

phases = trial.get_stance_swing_phases(side='Left')
# Use the first phase for the splines
phase = phases[0]
stance_phase = phase[:2]
swing_phase = phase[1:]
stance_times = [event.get_time(trial.points.rate) for event in stance_phase]
swing_times = [event.get_time(trial.points.rate) for event in swing_phase]

# Trim the dataframe to the stance and swing times
stance_df = df.filter(pl.col('time').is_between(stance_times[0], stance_times[-1]))
swing_df = df.filter(pl.col('time').is_between(swing_times[0], swing_times[-1]))

# Spline each to 100 points
num_points = 100
stance_times_np = stance_df['time'].to_numpy()
swing_times_np = swing_df['time'].to_numpy()
stance_times_spline = np.linspace(stance_times_np.min(), stance_times_np.max(), num_points)
swing_times_spline = np.linspace(swing_times_np.min(), swing_times_np.max(), num_points)

cs = CubicSpline(stance_times_np, stance_df.drop('time').to_numpy(), axis=0)
stance_spline = cs(stance_times_spline)
cs = CubicSpline(swing_times_np, swing_df.drop('time').to_numpy(), axis=0)
swing_spline = cs(swing_times_spline)
# Create a new dataframe with the splined data
stance_spline_df = pl.DataFrame({
    'time': stance_times_spline,
    **{f'{col}': stance_spline[:, i] for i, col in enumerate(stance_df.columns[1:])}
})
swing_spline_df = pl.DataFrame({
    'time': swing_times_spline,
    **{f'{col}': swing_spline[:, i] for i, col in enumerate(swing_df.columns[1:])}
})
# Concatenate the two dataframes vertically so that the time column is monotonically increasing
spline_df = pl.concat([stance_spline_df, swing_spline_df]).sort('time')

# Plot the splined data
spline_df.plot.line(x='time', y='ankle_l_flx_moment')


alt.Chart(...)

In [ ]:
# Build pathology groups
classification = 'AFIRM'
NR = [
    "LGS09",
    "LGS10",
    "LGS13",
    "LGS14",
    "E17",
    "E20",
    "E21",
    "LGS11",
    "LGS12",
    "LGS15",
    "LGS16",
    "T05",
    "T06",
    "E16",
    "E18",
    "E19",
    "LGS08",
    "T08",
    "E08",
    "E12",
    "E13",
    "E14",
    "E15",
    "E26",
    "LGS05",
    "LGS06",
    "LGS07",
    "T07"
]
TEMR = [
    "T01",
    "T02",
    "T03",
    "T04",
    "T09",
    "T10",
    "T11",
    "T12",
    "G14_E23",
    "E09",
    "E10",
    "A04",
    "A05",
    "A06",
    "A07",
    "A08"
]
HH = [
    "G01",
    "G02",
    "G03",
    "G04",
    "G05",
    "G06",
    "G07",
    "G08",
    "LGSH01",
    "LGSH02",
    "LGSH03",
    "LGSH04",
    "G13",
    "G14",
    "G15",
    "G16",
    "G17",
    "G18"
]
TEMR_HH = [
    "H01",
    "H02",
    "H03",
    "H04",
    "H05",
    "H06",
    "H07",
    "H08",
    "G16_E24",
    "E02",
    "E06",
    "E11",
    "A09",
    "A10",
    "A11",
    "A12"
]
HS = [
    "S01",
    "S03",
    "S04",
    "S05",
    "S06",
    "S07",
    "S08",
    "S13",
    "S09",
    "S10",
    "S11",
    "S12",
    "S14",
    "S15",
    "S16",
    "S17"
]
TEMR_KG = [
    "K01",
    "K02",
    "K03",
    "K04",
    "K05",
    "K06",
    "K07",
    "K08",
    "G13_E22",
    "E01",
    "E03",
    "E04",
    "E05",
    "A01",
    "A02",
    "A03"
]

groups = {
    "NR": NR,
    "TEMR": TEMR,
    "HH": HH,
    "TEMR_HH": TEMR_HH,
    "HS": HS,
    "TEMR_KG": TEMR_KG
}
